## Over parameterized network Non decreasing lr result code (not ran exp)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
def get_exp_details(root_dir, path):
    result_details={}
    details_path = root_dir+ 'details_'+ path + '.txt'
    with open(details_path, 'r') as f:
        lines = f.readlines()
    for line in lines:
        key, val = line[:-1].split(':')
        key, val = key.strip(' '), val.strip(' ')
        if key in ['ratio', 'alpha_0']:
            val = float(val)
        if key in ['Times', 'Weights', 'Epochs', 'book_keep_freq', 'g_times', 'g_epochs', 'freq_reduce_by', 'freq_reduce_after']:
            val = int(val)
        elif key in ['g_weight']:
            print('weight:', val)
            val = [int(t) for t in val[1:-1].split(', ')]
        result_details[key] = val
    return result_details

def get_exp_results(r_det, train_size=20000):
    root_dir = r_det['result_root_dir']
    path = r_det['result_path']
    grad_file_path = root_dir+'grad_'+path
    hess_file_path = root_dir+'hess_'+path
    gen_file_path  = root_dir+'gen_'+path
    
    # print(result_details)
    grad_list    = []
    hess_list    = []
    gen_err_list = []
    
    with open(grad_file_path, 'r') as f:
        lines = f.readlines()
    for line in lines:
        tensors = line[:-1].split(') ')
        # print('tensors',tensors)
        t_list = [float(t[7:].split(' ')[0][:-1]) for t in tensors]
        # print('grad_list_i length:',len(t_list))
        grad_list.append(t_list)
    
    with open(hess_file_path, 'r') as f:
        lines = f.readlines()
    for line in lines:
        tensors = line[:-1].split(' ')
        # print('tensors',tensors)
        t_list = [float(t[7:-1]) for t in tensors]
        # print('hess_list_i length:',len(t_list))
        hess_list.append(t_list)
    hess_list = np.array(hess_list)
    grad_list = np.array(grad_list)
    print('hess list shape:',hess_list.shape)
    print('grad list shape:',grad_list.shape)
    K_g = np.max(np.mean(np.array(grad_list), 0))
    L_g = np.max(np.mean(np.array(hess_list), 0))
    
    book_keep_freq = r_det['book_keep_freq']
    freq_reduce_after = r_det['freq_reduce_after']
    freq_reduce_by = r_det['freq_reduce_by']
    x_values = []
    count_keep = 0
    for i in range(train_size):
        if i%book_keep_freq==0:
            x_values.append(i+1)
            count_keep+=1
            if count_keep%freq_reduce_after==0:
                book_keep_freq+=freq_reduce_by
    
    return grad_list, hess_list, K_g, L_g, x_values

def plot_exp_data(root_dir_list, path_list, weights_list, save_fig=False, train_size=20000):
    grad_mean_list = []
    grad_df_list = []
    grad_err_list  = []
    hess_mean_list = []
    hess_df_list = []
    hess_err_list  = []
    data = {}
    x_values = []
    for root_dir, path, weights in zip(root_dir_list, path_list, weights_list):
        print('for weight:', weights)
        res_details = get_exp_details(root_dir, path)
        grad_list, hess_list, K_g, L_g, x_values = get_exp_results(res_details, train_size)
        grad_list_m = np.max(grad_list,1)
        hess_list_m = np.max(hess_list,1)
        # K_g = np.percentile(np.mean(grad_list, 0),90)
        # L_g = np.percentile(np.mean(hess_list, 0),90)
        
        # K_g = np.max(np.mean(grad_list, 1))
        # L_g = np.max(np.mean(hess_list, 1))
        # print('grad_list shape',grad_list_m.shape)
        # print('grad_list ',grad_list_m)
        # print(f'K_g:{K_g}, L_g:{L_g}')
        grad_mean, grad_err = np.mean(grad_list_m), np.std(grad_list_m)
        hess_mean, hess_err = np.mean(hess_list_m), np.std(hess_list_m)
        grad_mean_list.append(grad_mean)
        grad_err_list.append(grad_err)
        hess_mean_list.append(hess_mean)
        hess_err_list.append(hess_err)
        params_size = 784
        for weight in weights:
            params_size*= weight
        x_values.append(str(params_size))
        for run_id in range(len(grad_list)):
            for epoch in range(len(grad_list[run_id])):
                grad_df_list.append({'key':int(params_size), 'run_id': run_id, 'epoch':epoch, 'val': grad_list[run_id][epoch]})
                                      
        for run_id in range(len(hess_list)):
            for epoch in range(len(hess_list[run_id])):
                hess_df_list.append({'key':int(params_size), 'run_id': run_id, 'epoch':epoch, 'val': hess_list[run_id][epoch]})
        
        # plt.errorbar(x_values, np.mean(grad_list,0),np.std(grad_list,0), label='ratio:'+str(ratio))
    df1 = pd.DataFrame(grad_df_list)
    df2 = pd.DataFrame(hess_df_list)
    
    temp_df1 = df1.groupby(['key','run_id'])['val'].max().reset_index()
    temp_df2 = df2.groupby(['key','run_id'])['val'].max().reset_index()
    # temp_df1 = df1
    # temp_df2 = df2
    grad_mean_list = np.array(grad_mean_list)
    grad_err_list = np.array(grad_err_list)
    hess_mean_list = np.array(hess_mean_list)
    hess_err_list = np.array(hess_err_list)
    # x_values = [str(int(train_size*t)) for t in ratios]
    fig, ax = plt.subplots(figsize=(12,7))
    sns.pointplot(data = temp_df1, x='key',y='val', estimator=np.mean, ax=ax, label='Lipschitz const.', color='orange', errwidth=0)
    ax.fill_between(x_values, grad_mean_list-grad_err_list, grad_mean_list+ grad_err_list, color='orange', alpha=.3)
    plt.xlabel('Number of parameters', fontsize = 24)
    plt.ylabel('Lipschitz constant', fontsize = 24)    
    sns.set_context("talk")
    sns.set(font='sans-serif', style='whitegrid', font_scale=2, rc={"lines.linewidth": 2, "lines.markersize":10})
    plt.title('Mnist dataset', fontsize=24)
    if save_fig:
        plt.savefig('e3_Lipschitz_constant_mnist.png')
    
    fig, ax = plt.subplots(figsize=(12,7))
    sns.pointplot(data = temp_df2, x='key',y='val', ax=ax, label='Smoothness const.', color='green', errwidth=0)
    ax.fill_between(x_values, hess_mean_list-hess_err_list, hess_mean_list+ hess_err_list, color='green', alpha=.3)
    plt.xlabel('Number of parameters', fontsize = 24)
    plt.ylabel('Smoothness constant', fontsize = 24)    
    sns.set_context("talk")
    sns.set(font='sans-serif', style='whitegrid', font_scale=2, rc={"lines.linewidth": 2, "lines.markersize":10})
    # plt.show()
    plt.title('Mnist dataset', fontsize=24)
    if save_fig:
        plt.savefig('e3_Smoothness_constant_mnist.png')
    return df1, df2

In [2]:
root_dir_list = ['results/t0/','results/t0/','results/t0/']
path_list = ['try1_t8_w12', 'try1_t8_w16', 'try1_t8_w25']
ratios= [4., 6., 10.0, 1]
weights_list = [[12], [16], [25]]
grad_df, hess_f= plot_exp_data(root_dir_list[:], path_list[:], weights_list[:], save_fig=False)

for weight: [12]
weight: [12]
hess list shape: (0,)
grad list shape: (0,)


/home/harsh/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/harsh/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


AxisError: axis 1 is out of bounds for array of dimension 1